# RAG SETUP - Assignment 1



##1. Install libraries

In [ ]:
!pip install langchain-community langchain-openai faiss-cpu tiktoken pypdf ragas sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.

##2.Import required Python Packages

In [ ]:
  # 1. Imports
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_openai import AzureChatOpenAI
from ragas.metrics import LLMContextRecall
from ragas import EvaluationDataset
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall
from ragas.metrics import LLMContextPrecisionWithoutReference
from ragas.metrics import ResponseRelevancy
from ragas.metrics import Faithfulness
from ragas import evaluate

import os

##3. Import Azure OpenAI Keys & Endpoint

In [ ]:
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = ""

##4. Setup LLM with required parameters like temperature , top_p, max_tokens etc

In [ ]:
llm = AzureChatOpenAI(
    openai_api_version="2025-01-01-preview",
    azure_deployment="gpt-4.1-mini",  # Replace with your deployment name
    temperature=0.1,
    top_p=1,
    max_tokens=2000)

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant",
    ),
    ("human", "what is capital of india?"),
]
llm.invoke(messages)

AIMessage(content='The capital of India is New Delhi.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 22, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4f3d32ad4e', 'id': 'chatcmpl-C32IgU4EDEPButHyEsQK1nizVYfWL', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe

##5. Build FAISS Vector Index by loading the PDF & chunking , Embedding them

### Uplaod the 3 PDF
### 1. BNS.pdf
### 2. BNSS.pdf
### 3. BNSA.pdf

You can upload files directly to your Colab environment using the file icon in the left sidebar.

In [ ]:
# ----------------------
# Step 2: Load or Build FAISS Index
# ----------------------
vectorstore_path = "faiss_index"

if os.path.exists(vectorstore_path):
    print("🔄 Loading existing FAISS index...")
    embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    db = FAISS.load_local(vectorstore_path, embeddings=embed_model, allow_dangerous_deserialization=True)
else:
    print("🆕 Creating new FAISS index from PDFs...")
    loader = DirectoryLoader("data/", glob="**/*.pdf", loader_cls=PyPDFLoader)
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
    chunks = splitter.split_documents(docs)

    embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    db = FAISS.from_documents(chunks, embed_model)

    db.save_local(vectorstore_path)

🔄 Loading existing FAISS index...


/tmp/ipython-input-1031212661.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

##6.Retriveal Augment QA

In [ ]:

# ----------------------
# Step 3: Retrieval-Augmented QA
# ----------------------
retriever = db.as_retriever(search_kwargs={"k": 4})

prompt = ChatPromptTemplate.from_template("""
You are a legal assistant. Use the below excerpts from law documents to answer the question.
Mention source and page number after each answer fragment.

{context}

Question: {input}

Helpful Answer with citations:
""")

stuff_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriever, stuff_chain)



In [ ]:
# ----------------------
# Step 3: Retrieval-Augmented QA
# ----------------------
retriever = db.as_retriever(search_kwargs={"k": 4})

prompt = ChatPromptTemplate.from_template("""
You are a legal assistant. Use the below excerpts from law documents to answer the question.
Mention source and page number after each answer fragment.

{context}

Question: {input}

Helpful Answer with citations:
""")

stuff_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriever, stuff_chain)

In [ ]:
# ----------------------
# Step 3: Retrieval-Augmented QA
# ----------------------
retriever = db.as_retriever(search_kwargs={"k": 4})

prompt = ChatPromptTemplate.from_template("""
You are an expert legal assistant well-versed in Indian criminal law, specializing in the Bharatiya Nyaya Sanhita (BNS), Bharatiya Nagarik Suraksha Sanhita (BNSS), and Bharatiya Sakshya Adhiniyam (Indian Evidence Act) 2023. These three laws are foundational to handling all criminal cases in India. Your role is to assist judges, lawyers, and police officers by providing precise legal information and interpretations based on these laws, offering case-specific guidance.

The legal documents you are referencing consist of:
- **Bharatiya Nyaya Sanhita (BNS)**: The official criminal code in India that replaced the Indian Penal Code (IPC) on July 1, 2024.
- **Bharatiya Nagarik Suraksha Sanhita (BNSS)**: The main legislation detailing procedural law for administration of criminal justice, replacing the Criminal Procedure Code (CrPC).
- **Bharatiya Sakshya Adhiniyam, 2023**: The Indian Evidence Act, governing how evidence is to be treated in criminal cases.

When providing legal analysis or answering queries, do the following:
1. **Use relevant sections of the BNS, BNSS, and Bharatiya Sakshya Adhiniyam** based on the specific scenario being queried.
2. Ensure the legal provisions, sections, and terminologies you use are from the 2023 versions of the law (not outdated versions).
3. Provide clear explanations of legal terms, procedures, and how they apply to the scenario.
4. Offer interpretations that reflect recent changes in the law to guide decision-making.
5. If the user needs to explore similar cases or refer to legal precedents, offer summaries of related sections and guidelines.

{context}

Question: {input}

Helpful Answer with citations:
""")

stuff_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriever, stuff_chain)

##7.Ask Question & Get Answer

In [ ]:
# ----------------------
# Step 4: Ask a Question
# ----------------------
response = rag_chain.invoke({"input": "What is the section no that deals with  theft and punishment duration for the same  under the new law BNS?"})

In [ ]:
print("\n📘 Answer:\n", response["answer"])


📘 Answer:
 The section that deals with theft and its punishment under the new law BNS is **Section 304 - Snatching**. According to this section, theft is defined as snatching if the offender suddenly or forcibly takes away movable property from any person. The punishment for snatching (theft) is imprisonment of either description for a term which may extend to three years, and the offender shall also be liable to a fine (Source: page 78).

Additionally, **Section 307** addresses theft committed after preparation for causing death, hurt, or restraint to any person in order to commit theft or to escape after committing theft. The punishment under this section is rigorous imprisonment for a term which may extend to ten years, along with a fine (Source: page 88).

Therefore, the relevant sections for theft and punishment duration under the new law BNS are:
- Section 304: Imprisonment up to 3 years and fine (page 78)
- Section 307: Rigorous imprisonment up to 10 years and fine (page 88)


##8. Evaluate the Results using a pre-curated Synthetic Golden Dataset

In [ ]:
EVAL_QUESTIONS=[
    "What are the section that deal with bribery?",
    "What is the punishemnt for kidnapping or abducting in order to murder or for ransom?",
    "What is the section that deals with culpable homicide?",
    "What is the section that deals with donwry?"
]

EVAL_ANSWERS=[
    "Section 170 deals with bribery, diefining it is a giving or accepting gratification to induce someone to exercise electoral rights.Section 173 addresses punishment for bribery, stating it shall be punished for fine only.",
    "For fidnapping or abducting in order to murder the punishment is imprisonment for life or rigorous imprisonment for 10 years and fine.For kidnapping for ransome the punishment is death or imprisonment for life and fine. Both offenses are cognizable and tried by the cour of session",
    "Section 100 deals with culpable homicide.Section 101 defines murder, while section 105 addresses punishment for culpable homicide not amounting to murder, Section 102 covers coulpable homicide by causing the death of a person other than the intended victim",
    "Section 80 deals with dowry death. It defines dowry death as the death of a woman with seven years of marriage caused by burns, bodily injury or under abnormal circumstances where she was subjected to cruelty or harassement related to downry demands"
]
dataset =[]

for query, reference in zip(EVAL_QUESTIONS, EVAL_ANSWERS):
  relevant_docs = retriever.invoke(query)
  response = rag_chain.invoke({"input": query})
  dataset.append({"user_input": query,
                  "retrieved_contexts": [rdoc.page_content for rdoc in relevant_docs],
                  "reference_answer": reference,
                  "response": response["answer"],
                  "reference":reference
                  })

In [ ]:
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [ ]:
evaluator_llm = LangchainLLMWrapper(llm)

In [ ]:
result = evaluate(dataset =evaluation_dataset,
                  metrics =[LLMContextRecall(),Faithfulness(),ResponseRelevancy(),LLMContextPrecisionWithoutReference()],
                  llm = evaluator_llm, embeddings= embed_model )

Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
print(result)

{'context_recall': 0.4792, 'faithfulness': 0.9286, 'answer_relevancy': 0.7317, 'llm_context_precision_without_reference': 0.7708}
